## nomad-parser-nexus demo for Multidimensional Photoemission Spectroscopy (MPES)

Jupyter notebook displaying parsing from FHI preprocessing software output (x-array based .h5) to NXmpes formatted nexus

### **Step -1:** Create a new virtual environment and set up dependencies for jupyterlab_h5web.<br> This section **should only be run if you do not have Jupyter Lab, its extensions and the extra needed packages.** <br>  For use within the Nomad UI this section should be skipped.<br> These cells can be run in a Jupyter notebook or copied to a terminal without the "!".

Run the following to start with a fresh virtualenv in **your terminal** in which you are installing and then running **Jupyter lab** if you wish to: <br>
```pip install virtualenv && virtualenv --python=python3.7 .nexusenv && source .nexusenv/bin/activate```

Install jupyter, jupyter-lab and h5web extensions.

In [ ]:
! pip install --upgrade nodejs && pip install ipywidgets h5py==3.5.0 h5glance==0.7 h5grove==0.0.14 jupyterlab[full]==3.2.9 jupyterlab_h5web[full]==1.3.0 punx==0.2.5 nexpy==0.14.1 silx[full]==1.0.0 && jupyter lab build

Enable the extensions

In [ ]:
! jupyter nbextension enable --py widgetsnbextension

In [ ]:
! jupyter serverextension enable jupyterlab_h5web

### **Step 0:** Installing and testing nomad-parser-nexus module. <br> This section **should only be run if you are not running this within NOMAD.** 

Install nomad and its dependencies. Do not run the following cell if you have a nomad installation running. 

In [ ]:
! pip install --upgrade pip && pip install nomad-lab==1.0.0 --extra-index-url https://gitlab.mpcdf.mpg.de/api/v4/projects/2187/packages/pypi/simple
! pip list

Install mpes and its dependecies

In [ ]:
! pip install git+https://github.com/rettigl/mpes.git
! pip list

Install the nexusparser and its requirements

In [ ]:
! git clone https://github.com/Arora0/nomad-parser-nexus.git --recursive && cd nomad-parser-nexus && git checkout dataconverter-baseclasses-check --recurse-submodules && git status && pip install -r requirements.txt && pip install -e .[all]

The following cell restarts the kernel after the nexusparser installation

In [ ]:
import os
os._exit(00)

### **Step 1:** Download example RAW data (trARPES data of WSe2)

Check dependencies of nomad and of the nexusparser and prints them. If the nexusparser and nomad-lab are installed, you are ready to go.<br> Check if jupyterlab_h5web server and lab extensions are enabled and OK. 

In [ ]:
! pip list | grep 'nomad\|nexus'
! jupyter serverextension list
! jupyter labextension list

Download mpes example data from Zenodo

In [ ]:
import shutil
! curl --output WSe2.zip https://zenodo.org/record/6369728/files/WSe2.zip
shutil.unpack_archive('WSe2.zip')

In [ ]:
from mpes import base, fprocessing as fp, analysis as aly
import matplotlib.pyplot as plt
import numpy as np
import os
from dask import compute
import datetime as dt
import h5py
fdir = r'Scan049_1'
outdir = r'Scan049_binned/'
if not os.path.exists(outdir):
    os.makedirs(outdir)

Initial data binning for distortion correction

In [ ]:
parp = fp.parallelHDF5Processor(folder=fdir)
parp.gather(identifier=r'/*.h5', file_sorting=True)
len(parp.files)

In [ ]:
parp.files = parp.files[0:50]
axes = ['X', 'Y', 't']
# Important to keep the whole detector area for the initial binning!
bins = [512, 512, 300]
ranges = [(0, 2048), (0, 2048), (64000, 68000)]
parp.parallelBinning(axes=axes, nbins=bins, ranges=ranges, scheduler='threads', ret=False)

Determine correction landmarks

In [ ]:
mc = aly.MomentumCorrector(parp.combinedresult['binned'])
mc.selectSlice2D(slice(165, 175), 2)
mc.featureExtract(mc.slice, sigma=5, fwhm=10, sigma_radius=3)
mc.view(points=mc.features, annotated=True)

Calculate thin plate spline symmetry correction

In [ ]:
mc.splineWarpEstimate(image=mc.slice, landmarks=mc.pouter_ord, include_center=True,
                      iterative=False, interp_order=2, update=True)
mc.view(image=mc.slice_transformed, annotated=True, points={'feats':mc.ptargs}, backend='bokeh', crosshair=True, radii=[75,110,150], crosshair_thickness=0.2)

Image registration

In [ ]:
mc.coordinateTransform(type='translation', xtrans=70, ytrans=70, keep=True)
plt.imshow(mc.slice_transformed, origin='lower', cmap='terrain_r')
plt.axvline(x=256)
plt.axhline(y=256)

In [ ]:
mc.coordinateTransform( type='rotation', angle=-5, center=(256., 256.), keep=True)
plt.imshow(mc.slice_transformed, origin='lower', cmap='terrain_r')
plt.axvline(x=256)
plt.axhline(y=256)

In [ ]:
# Final Deformation field:
subs = 20
plt.scatter(mc.cdeform_field[::subs,::subs].ravel(), mc.rdeform_field[::subs,::subs].ravel(), c='b')

Momentum calibration

In [ ]:
# Pick one high-symmetry point
point_b = [252.,255.]
# Pick the BZ center
point_a = [308.,346.]
# give the distance of the two in inverse Angstrom
distance = np.pi*4/3/3.297
# Momentum calibration assuming equal scaling along both x and y directions (equiscale=True)
# Requirements : pixel coordinates of and the momentum space distance between two symmetry points, plus the momentum coordinates
# of one of the two points 
ext = mc.calibrate(mc.slice_transformed, point_from=point_a, point_to=point_b, dist=distance, equiscale=True, ret=['extent'])

In [ ]:
mc.view(image=mc.slice_transformed, imkwds=ext)
plt.xlabel('$k_x$', fontsize=15)
plt.ylabel('$k_y$', fontsize=15)

Energy calibration

In [ ]:
axes = ['t']
bins = [1000]
ranges = [(63000, 80000)]
traces, tof = fp.extractEDC(folder=r'energycal_2019_01_08', axes=axes, bins=bins, ranges=ranges)

In [ ]:
voltages = np.arange(-12.2, -23.2, -1)
ec = aly.EnergyCalibrator(biases=voltages, traces=traces, tof=tof)

In [ ]:
ec.normalize(smooth=True, span=7, order=1)
ec.view(traces=ec.traces_normed, xaxis=ec.tof, backend='bokeh')

In [ ]:
rg = [(65000, 65200)]
ec.addFeatures(traces=ec.traces_normed, refid=0, ranges=rg[0], infer_others=True, mode='append')
ec.featranges

In [ ]:
ec.featureExtract(traces=ec.traces_normed, ranges=ec.featranges)
ec.view(traces=ec.traces_normed, peaks=ec.peaks, backend='bokeh')

Calculate energy calibration

In [ ]:
refid=5
Eref=-1.3
axs = ec.calibrate(ret='all', Eref=Eref, t=ec.tof, refid=refid)
ec.view(traces=ec.traces_normed, xaxis=ec.calibration['axis'], backend='bokeh')

Quality of calibration

In [ ]:
for i in range(0,len(voltages)):
    plt.plot(ec.calibration['axis']-(voltages[i]-voltages[refid]), ec.traces_normed[i])
plt.xlim([-15,5])

Inspect calibration function

In [ ]:
ec.view(traces=ec.calibration['axis'][None,:], xaxis=ec.tof, backend='matplotlib', show_legend=False)
plt.scatter(ec.peaks[:,0], ec.biases-ec.biases[refid]+Eref, s=50, c='k')
plt.xlabel('Time-of-flight', fontsize=15)
plt.ylabel('Energy (eV)', fontsize=15)
plt.ylim([-8,6])
plt.xlim([63400,69800])

Dataframe processor

In [ ]:
dfp = fp.dataframeProcessor(datafolder=fdir)
# Read events in with ms time stamps
dfp.read(source='folder', ftype='h5', timeStamps=True)

Apply energy calibration

In [ ]:
dfp.appendEAxis(E0=ec.calibration['E0'], a=ec.calibration['coeffs'])
dfp.edf.head(8)

Apply distortion correction

In [ ]:
dfp.applyKCorrection(type='tps_matrix', rdeform_field = mc.rdeform_field, cdeform_field = mc.cdeform_field, X='X', Y='Y', newX='Xm', newY='Ym')
dfp.edf.head(8)

Apply momentum calibration

In [ ]:
dfp.appendKAxis(point_b[0], point_b[1], X='Xm', Y='Ym', rstart=parp.binranges[0][0], cstart=parp.binranges[1][0], \
                rstep=parp.binsteps[0], cstep=parp.binsteps[1], fc=mc.calibration['coeffs'][0], fr=mc.calibration['coeffs'][1])
dfp.edf.head(8)

Apply pump-probe delay axis conversion

In [ ]:
ADCRange = (650, 6900)
timeRange = (-100, 200)
dfp.edf['delay'] = timeRange[0] + (dfp.edf['ADC']-ADCRange[0]) * (timeRange[1] - timeRange[0])/(ADCRange[1]-ADCRange[0])
dfp.edf.head(8)

Bin 4D data in transformed grid

In [ ]:
axes = ['kx', 'ky', 'E', 'delay']
bins = [50, 50, 100, 21]
ranges = [(-2, 2), (-2, 2), (-3, 2), (-110, 190)]
# jittering of energy and ADC should best be done on the bin size of the hardware, not the rebinned bin size
TOFrange=[64500,67000]
e_t_conversion = (base.tof2evpoly(ec.calibration['coeffs'], ec.calibration['E0'], TOFrange[0]) - base.tof2evpoly(ec.calibration['coeffs'], ec.calibration['E0'], TOFrange[1]))/(TOFrange[1]-TOFrange[0])
d_adc_conversion = (timeRange[1]-timeRange[0])/(ADCRange[1]-ADCRange[0])
jitter_amplitude = [0.5, 0.5, 1*bins[2]/abs(ranges[2][1]-ranges[2][0])*e_t_conversion, 1*bins[3]/abs(ranges[3][1]-ranges[3][0])*d_adc_conversion]
dfp.distributedBinning(axes=axes, nbins=bins, ranges=ranges, scheduler='threads', ret=False, jittered=True, jitter_amplitude=jitter_amplitude)

Create metatada structure

In [ ]:
metadata = {}

In [ ]:
# Get time stamps from dataframe
dfpart = dfp.edf.get_partition(0)
all_data = np.array(compute(dfpart.values))[0,:,:]
timeStamps = all_data[:,6]
tsFrom = timeStamps[0]
dfpart = dfp.edf.get_partition(len(dfp.datafiles)-1)
all_data = np.array(compute(dfpart.values))[0,:,:]
timeStamps = all_data[:,6]
tsTo = timeStamps[len(timeStamps)-1]
metadata['timing'] = {'acquisition_start': dt.datetime.utcfromtimestamp(tsFrom/1000).replace(tzinfo=dt.timezone.utc).isoformat(),
                      'acquisition_stop': dt.datetime.utcfromtimestamp(tsTo/1000).replace(tzinfo=dt.timezone.utc).isoformat(),
                      'acquisition_duration': int((tsTo - tsFrom)/1000),
                      'bin_array_creation': dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc).isoformat(),
                      'collection_time': float((tsTo - tsFrom)/1000)
                      }

In [ ]:
#import meta data from data file
file0 = parp.files[0]

metadata['file'] = {}
with h5py.File(file0, 'r') as f:
    for k,v in f.attrs.items():
        metadata['file'][k]=v      
        
metadata['file']['KTOF:Lens:Extr:I'] = 0.
metadata['file']['KTOF:Lens:UDLD:VSet'] = 400.
metadata['file']['KTOF:Lens:Sample:VSet'] = 17.

metadata['entry_identifier'] = fdir[13:-2]

In [ ]:
#Meta data of the binning
momentum_dict = mc.__dict__.copy()
energy_dict = ec.__dict__.copy()
binning_dict = dfp.__dict__.copy()

momentum_dict['calibration']['coeffs'] = np.array(momentum_dict['calibration']['coeffs'])
momentum_dict['adjust_params']['center'] = np.array(momentum_dict['adjust_params']['center'])
momentum_dict['pcent'] = np.array(momentum_dict['pcent'])
# to reduce the size of h5 file for upload on github
momentum_dict.pop('image')
binning_dict.pop('histdict')
binning_dict.pop('dfield')

metadata['energy_correction'] = energy_dict
metadata['momentum_correction'] = momentum_dict
metadata['binning'] = binning_dict

In [ ]:
# manual Meta data
#General
metadata['experiment_summary'] = 'WSe2 XUV NIR pump probe data.'
metadata['entry_title'] = 'Valence Band Dynamics - 800 nm linear s-polarized pump, 0.6 mJ/cm2 absorbed fluence'
metadata['experiment_title'] = 'Valence band dynamics of 2H-WSe2'

#User
# Fill general parameters of NXuser
# TODO: discuss how to deal with multiple users?
metadata['user0'] = {}
metadata['user0']['name'] = 'Julian Maklar'
metadata['user0']['role'] = 'Principal Investigator'
metadata['user0']['affiliation'] = 'Fritz Haber Institute of the Max Planck Society'
metadata['user0']['address'] = 'Faradayweg 4-6, 14195 Berlin'
metadata['user0']['email'] = 'maklar@fhi-berlin.mpg.de'

#NXinstrument
metadata['instrument'] = {}
#analyzer
metadata['instrument']['analyzer']={}
metadata['instrument']['analyzer']['slow_axes'] = "delay" # the scanned axes
metadata['instrument']['analyzer']['spatial_resolution'] = 10.
metadata['instrument']['analyzer']['energy_resolution'] = 110.
metadata['instrument']['analyzer']['momentum_resolution'] = 0.08
metadata['instrument']['analyzer']['projection'] = "reciprocal"
metadata['instrument']['analyzer']['working_distance'] = 4.
metadata['instrument']['analyzer']['lens_mode'] = "6kV_kmodem4.0_30VTOF.sav"

# Need to get those from the data file...
# Put into separate Lens objects?
metadata['instrument']['analyzer']['fa_size'] = 200
metadata['instrument']['analyzer']['ca_size'] = np.nan

#probe beam
metadata['instrument']['beam']={}
metadata['instrument']['beam']['probe']={}
metadata['instrument']['beam']['probe']['incident_energy'] = 21.7
metadata['instrument']['beam']['probe']['incident_energy_spread'] = 0.11
metadata['instrument']['beam']['probe']['pulse_duration'] = 20.
metadata['instrument']['beam']['probe']['frequency'] = 500.
metadata['instrument']['beam']['probe']['incident_polarization'] = [1, 1, 0, 0] # p pol Stokes vector
metadata['instrument']['beam']['probe']['extent'] = [80., 80.] 
#pump beam
metadata['instrument']['beam']['pump']={}
metadata['instrument']['beam']['pump']['incident_energy'] = 1.55
metadata['instrument']['beam']['pump']['incident_energy_spread'] = 0.08
metadata['instrument']['beam']['pump']['pulse_duration'] = 35.
metadata['instrument']['beam']['pump']['frequency'] = 500.
metadata['instrument']['beam']['pump']['incident_polarization'] = [1, -1, 0, 0] # s pol Stokes vector
metadata['instrument']['beam']['pump']['incident_wavelength'] = 800. 
metadata['instrument']['beam']['pump']['average_power'] = 300.
metadata['instrument']['beam']['pump']['pulse_energy'] = metadata['instrument']['beam']['pump']['average_power']/metadata['instrument']['beam']['pump']['frequency']#µJ
metadata['instrument']['beam']['pump']['extent'] = [230., 265.] 
metadata['instrument']['beam']['pump']['fluence'] = 0.15

#sample
metadata['sample']={}
metadata['sample']['preparation_date'] = '2019-01-13T10:00:00+00:00'
metadata['sample']['sample_history'] = 'Cleaved'
metadata['sample']['chemical_formula'] = 'WSe2'
metadata['sample']['description'] = 'Sample'
metadata['sample']['name'] = 'WSe2 Single Crystal'
metadata['sample']['temperature'] = 300.
metadata['sample']['pressure'] = 5.e-11

Functions for generating xarrays from the data, and saving as intermediary h5 file

In [ ]:
import xarray as xr
default_units = {
    'X': 'step', 
    'Y': 'step', 
    't': 'step', 
    'tofVoltage':'V',
    'extractorVoltage':'V',
    'extractorCurrent':'A',
    'cryoTemperature':'K',
    'sampleTemperature':'K',
    'dldTimeBinSize':'ns',
    'delay':'fs',
    'timeStamp':'s',
    'E':'eV',
    'energy':'eV',
    'kx':'1/A',
    'ky':'1/A'}

def res_to_xarray(res, binNames, binAxes, metadata=None):
    """ creates a BinnedArray (xarray subclass) out of the given np.array
    Parameters:
        res: np.array
            nd array of binned data
        binNames (list): list of names of the binned axes
        binAxes (list): list of np.arrays with the values of the axes
    Returns:
        ba: BinnedArray (xarray)
            an xarray-like container with binned data, axis, and all available metadata
    """
    dims = binNames
    coords = {}
    for name, vals in zip(binNames, binAxes):
        coords[name] = vals

    xres = xr.DataArray(res, dims=dims, coords=coords)

    for name in binNames:
        try:
            xres[name].attrs['unit'] = default_units[name]
        except KeyError:
            pass

    xres.attrs['units'] = 'counts'
    xres.attrs['long_name'] = 'photoelectron counts'

    if metadata is not None:
        xres.attrs['metadata'] = metadata

    return xres

import h5py
def xarray_to_h5(data, faddr, mode='w'):
    """ Save xarray formatted data to hdf5
    Args:
        data (xarray.DataArray): input data
        faddr (str): complete file name (including path)
        mode (str): hdf5 read/write mode
    Returns:
    """
    with h5py.File(faddr, mode) as h5File:

        print(f'saving data to {faddr}')

        # Saving data

        ff = h5File.create_group('binned')

        # make a single dataset
        ff.create_dataset('BinnedData', data=data.data)

        # Saving axes
        aa = h5File.create_group("axes")
        # aa.create_dataset('axis_order', data=data.dims)
        ax_n = 0
        for binName in data.dims:
            ds = aa.create_dataset(f'ax{ax_n}', data=data.coords[binName])
            ds.attrs['name'] = binName
            ax_n += 1


        if ('metadata' in data.attrs and isinstance(data.attrs['metadata'], dict)):
            meta_group = h5File.create_group('metadata')
            
            def recursive_write_metadata(h5group, node):
                for key, item in node.items():
                    if isinstance(item, (np.ndarray, np.int64, np.float64, str, bytes, int, float, list)):
                        try:
                            h5group.create_dataset(key, data=item)
                        except TypeError:
                            h5group.create_dataset(key, data=str(item))
                            print("saved " + key + " as string")
                    elif isinstance(item, dict):
                        print(key)
                        group = h5group.create_group(key)
                        recursive_write_metadata(group, item)
                    else:
                        try:
                            h5group.create_dataset(key, data=str(item))
                            print("saved " + key + " as string")
                        except:
                            raise ValueError('Cannot save %s type'%type(item))
            
            
     
            recursive_write_metadata(meta_group, data.attrs['metadata'])
                   
    print('Saving complete!')

Convert and save as h5 file

In [ ]:
axnames = dfp.binaxes.copy()
axnames[2] = "energy"
axes = [dfp.histdict[ax] for ax in dfp.binaxes]
res_xarray = res_to_xarray(dfp.histdict['binned'], axnames, axes, metadata)
xarray_to_h5(res_xarray, "WSe2_xarray.h5")

### **Step 2:** Run your mpes-specific parser on the example data

Set the nexusparser directory.

In [ ]:
import os
import nexusparser
nexus_dir = os.path.dirname(nexusparser.__file__)  # where the nexusparser module is located!!!!
print(nexus_dir)

Now we run our parser. The --reader flag takes the mpes reader (mpes), the --nxdl flag takes the application definition for this technique.<br> 

In [ ]:
! python "{nexus_dir}/tools/dataconverter/convert.py" \
--reader mpes \
--nxdl NXmpes \
--input-file WSe2_xarray.h5 \
--input-file config_file.json \
--input-file ELN_metadata_example.yaml \
--output WSe2.mpes.nxs

The key take home message is that the command above-specified triggers the automatic creation of the HDF5 file.<br>
This *.nxs file, is an HDF5 file.

### Run the same parser alternatively using a function

In [ ]:
from nexusparser.tools.dataconverter.convert import convert

In [ ]:
convert(input_file=["config_file.json","ELN_metadata_example.yaml"],
        reader='mpes',
        nxdl='NXmpes',
        output='WSe2.mpes.nxs',
        objects=(res_xarray))

### **Step 3:** Inspect the HDF5/NeXus file WSe2.mpes.nxs using H5Web

In [ ]:
from jupyterlab_h5web import H5Web

In [ ]:
h5_file_name = 'WSe2.mpes.nxs'

In [ ]:
H5Web(h5_file_name)